In [3]:
### If after importing lightfm, you get this warning:
### "UserWarning: LightFM was compiled without OpenMP support. 
### Only a single thread will be used."
### run the following bash commands to ensure it optimizes its
### use of all available CPU cores (much faster training times)
!brew install gcc # now check version of gcc (!gcc-14 --version)
!CC=gcc-14 pip install --no-binary lightfm lightfm --force-reinstall

==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################### 100.0%
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
To reinstall 14.2.0, run:
  brew reinstall gcc
  Using cached lightfm-1.17-cp311-cp311-macosx_14_0_arm64.whl
  Using cached numpy-2.1.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached scipy-1.14.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached job

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
# all_cleaned = pd.read_csv('../data/all_cleaned.csv', usecols=['user_id', 'isbn', 'book_rating', 'mod_book_title', 'mod_book_author', 'genre'])


In [6]:
# all_cleaned.head()

In [4]:
%run import_data_.py

Continuing with existing version of data folder
Goodreads dataset loaded successfully as books_goodreads
Pandas dataframes (books_goodreads, books_big, book, users, ratings) loaded successfully
Columns in DataFrames 'users' and 'ratings' renamed
You can use the DataFrames 'books' or 'books_big' - they are exactly the same (big) dataset
loading books_ratings and books_users_ratings
Ready to go!


In [8]:
# Convert 'user_id' to string to ensure compatibility with LightFM
books_users_ratings['user_id'] = books_users_ratings['user_id'].astype(str)

In [9]:
books_users_ratings.head()

,isbn,book_title,book_author,user_id,individual_rating,avg_rating,nr_ratings,nr_readers,genre,year_of_publication,publisher,annotations,mod_title,mod_author,mod_publisher,image_url_s,image_url_m,image_url_l
0,590085417,Heidi,Johanna Spyri,102967.0,8.0,8.0,2,3,"['Juvenile Fiction', 'Fiction', ""Children's Li...",2021,Scholastic,Heidi (Fictitious character : Spyri),heidi,johanna spyri,scholastic,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...
1,590085417,Heidi,Johanna Spyri,111702.0,8.0,8.0,2,3,"['Juvenile Fiction', 'Fiction', ""Children's Li...",2021,Scholastic,Heidi (Fictitious character : Spyri),heidi,johanna spyri,scholastic,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...
2,590085417,Heidi,Johanna Spyri,135473.0,0.0,8.0,2,3,"['Juvenile Fiction', 'Fiction', ""Children's Li...",2021,Scholastic,Heidi (Fictitious character : Spyri),heidi,johanna spyri,scholastic,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...
3,68107468X,Edgar Allen Poe Collected Poems,Edgar Allan Poe,204964.0,8.0,8.0,2,2,['Poetry'],2020,Bausch & Lombard,NaN,edgar allen poe collected poems,edgar allan poe,bausch lombard,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...
4,68107468X,Edgar Allen Poe Collected Poems,Edgar Allan Poe,267830.0,8.0,8.0,2,2,['Poetry'],2020,Bausch & Lombard,NaN,edgar allen poe collected poems,edgar allan poe,bausch lombard,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...


In [10]:
books_users_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917916 entries, 0 to 917915
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 917916 non-null  object 
 1   book_title           917916 non-null  object 
 2   book_author          917916 non-null  object 
 3   user_id              917916 non-null  object 
 4   individual_rating    917916 non-null  float64
 5   avg_rating           917916 non-null  float64
 6   nr_ratings           917916 non-null  int64  
 7   nr_readers           917916 non-null  int64  
 8   genre                917916 non-null  object 
 9   year_of_publication  917916 non-null  int64  
 10  publisher            917916 non-null  object 
 11  annotations          438490 non-null  object 
 12  mod_title            917916 non-null  object 
 13  mod_author           917916 non-null  object 
 14  mod_publisher        917916 non-null  object 
 15  image_url_s      

This following cell has been commented out because, in the updated dataset books_users_ratings, the column 'genre' already consists of a list of string values which are the categorized genres according to the same code.

In [10]:
# # categorize genre

# # Define the genre mapping
# genre_mapping = {
#     'fiction': 'Fiction',
#     'romance': 'Romance',
#     'historical': 'Historical Fiction',
#     'thrillers': 'Thriller',
#     'suspense': 'Suspense',
#     'juvenile fiction': 'Juvenile Fiction',
#     'children\'s literature': 'Children\'s Literature',
#     'guidebooks': 'Guidebooks',
#     'poetry': 'Poetry',
#     'songs': 'Music',
#     'ballads': 'Music',
#     'unknown': 'Unknown',
#     'error fetching data': 'Unknown'
# }

# # Function to categorize genres
# def categorize_genre(genre):
#     genre_lower = genre.lower()
#     for keyword, category in genre_mapping.items():
#         if keyword in genre_lower:
#             return category
#     return 'Other'

# # Apply the function to the genre column
# all_cleaned['categorized_genre'] = all_cleaned['genre'].apply(categorize_genre)

# print(all_cleaned[['genre', 'categorized_genre']])

In [3]:
# dropping the genre column
# all_cleaned = all_cleaned.drop(columns=['genre'])
# all_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183600 entries, 0 to 183599
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   isbn               183600 non-null  object 
 1   user_id            183600 non-null  object 
 2   book_rating        183600 non-null  float64
 3   mod_book_author    183600 non-null  object 
 4   mod_book_title     183600 non-null  object 
 5   categorized_genre  183600 non-null  object 
dtypes: float64(1), object(5)
memory usage: 8.4+ MB


Bookmark for Lay: adjust this code for string list type 'genre'

In [9]:
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix, coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score
from sklearn.model_selection import train_test_split

# Function to build interactions as a sparse matrix
def build_interactions(df):
    user_ids = df['user_id'].astype('category').cat.codes.values
    item_ids = df['isbn'].astype('category').cat.codes.values
    ratings = df['individual_rating'].values
    return coo_matrix((ratings, (user_ids, item_ids)))

# Function to build item features as a sparse matrix
def build_item_features(df, feature_index_map):
    num_items = len(df['isbn'].unique())
    num_features = len(feature_index_map)
    feature_matrix = lil_matrix((num_items, num_features))

    item_cats = pd.Categorical(df['isbn'])
    item_codes = item_cats.codes

    for i, row in df.iterrows():
        item_idx = item_cats.codes[df['isbn'] == row['isbn']][0]
        features = [row['mod_author']] + (row['genre'] if isinstance(row['genre'], list) else [row['genre']])
        feature_indices = [feature_index_map.get(f) for f in features if feature_index_map.get(f) is not None]
        feature_matrix[item_idx, feature_indices] = 1

    return feature_matrix.tocsr()  # Convert to csr_matrix for efficient operations later

# Create feature index map
def create_feature_index_map(df):
    features = set()
    for _, row in df.iterrows():
        features.add(row['mod_author'])
        if isinstance(row['genre'], list):
            features.update(row['genre'])
        else:
            features.add(row['genre'])
    
    feature_index_map = {feature: idx for idx, feature in enumerate(sorted(features))}
    return feature_index_map

# Define LightFM model
model = LightFM(loss='warp')

# Prepare your data
# Assuming books_users_ratings is your DataFrame
X = books_users_ratings[['user_id', 'isbn']]
y = books_users_ratings['individual_rating']

# Split the data
train_data, test_data = train_test_split(books_users_ratings, test_size=0.2, random_state=42)

# Create feature index map
feature_index_map = create_feature_index_map(train_data)

# Build interaction matrices
train_interactions = build_interactions(train_data)
test_interactions = build_interactions(test_data)

# Build item features
item_features = build_item_features(train_data, feature_index_map)

# Training with early stopping
best_auc = 0
patience = 5
no_improve = 0
num_epochs = 30

for epoch in range(num_epochs):
    model.fit(train_interactions, item_features=item_features, epochs=1, num_threads=8)
    
    # Evaluate on the test set
    auc = auc_score(model, test_interactions, item_features=item_features).mean()
    print(f'Epoch {epoch + 1}: AUC = {auc}')
    
    if auc > best_auc:
        best_auc = auc
        no_improve = 0
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print('Early stopping')
        break

print(f'Final AUC: {best_auc}')


ValueError: buffer source array is read-only

In [8]:
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix, coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score


# Function to build interactions as a sparse matrix
def build_interactions(df):
    user_ids = df['user_id'].astype('category').cat.codes.values
    item_ids = df['isbn'].astype('category').cat.codes.values
    ratings = df['individual_rating'].values
    return coo_matrix((ratings, (user_ids, item_ids)))

# Function to build item features as a sparse matrix
def build_item_features(df, feature_index_map):
    num_items = len(df['isbn'].unique())
    num_features = len(feature_index_map)
    feature_matrix = lil_matrix((num_items, num_features))

    item_cats = pd.Categorical(df['isbn'])
    item_codes = item_cats.codes

    for i, row in df.iterrows():
        item_idx = item_cats.codes[df['isbn'] == row['isbn']][0]
        features = [row['mod_author']] + (row['genre'] if isinstance(row['genre'], list) else [row['genre']])
        feature_indices = [feature_index_map.get(f) for f in features if feature_index_map.get(f) is not None]
        feature_matrix[item_idx, feature_indices] = 1

    return feature_matrix.tocsr()  # Convert to csr_matrix for efficient operations later


# Create feature index map
def create_feature_index_map(df):
    features = set()
    for _, row in df.iterrows():
        features.add(row['mod_author'])
        if isinstance(row['genre'], list):
            features.update(row['genre'])
        else:
            features.add(row['genre'])
    
    feature_index_map = {feature: idx for idx, feature in enumerate(sorted(features))}
    return feature_index_map

# Define LightFM model
model = LightFM(loss='warp')

# Prepare your data
# Assuming books_users_ratings is your DataFrame
X = books_users_ratings[['user_id', 'isbn']]
y = books_users_ratings['individual_rating']

# Split the data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(books_users_ratings, test_size=0.2, random_state=42)

# Create feature index map
feature_index_map = create_feature_index_map(train_data)

# Build interaction matrices
train_interactions = build_interactions(train_data)
test_interactions = build_interactions(test_data)

# Build item features
item_features = build_item_features(train_data, feature_index_map)

# Train the LightFM model on the training fold
model.fit(train_interactions, item_features=item_features, epochs=30, num_threads=8)

# Evaluate the model using AUC score on the test fold
auc = auc_score(model, test_interactions, item_features=item_features).mean()

print(f'AUC: {auc}')


NameError: name 'item_features' is not defined

In [16]:

# LightFM model
###############(loss='warp') # 'warp' loss functions are good for finding relevant results 
                             # be it for implicit interactions (e.g. clicks or views)
                             # or explicit ones (ratings), and they're efficient
                             # Other options were:
                             # 1. 'logistic': slower and would give recommendations that the
                             # user is expected to like, but it may "overfit to their taste"
                             # causing a more bubble / echo chamber kind of recommendations
                             # 2. 'bpr': good for implicit interactions (not explicit ratings)
                             # so it would be a waste of the data we have for ratings



In [24]:
from lightfm import LightFM
from lightfm.data import Dataset

# Build the interaction matrix with user-item pairs
(interactions, _) = dataset.build_interactions(
    [(x[0], x[1]) for x in books_users_ratings[['user_id', 'isbn']].values]
)

# Retrieve the user-item pairs for ratings
interaction_tuples = [(x[0], x[1]) for x in books_users_ratings[['user_id', 'isbn']].values]

# Build weights array from individual ratings by matching interaction tuples
weights = np.array([books_users_ratings.loc[(books_users_ratings['user_id'] == u) & 
                                            (books_users_ratings['isbn'] == i), 
                                            'individual_rating'].values[0] 
                    for u, i in interaction_tuples])

# Create item features by combining author and genres
item_features = dataset.build_item_features(
    [(row['isbn'], [row['mod_author']] + row['genre']) for _, row in books_users_ratings.iterrows()]
)

# Define the LightFM model using the logistic loss function for explicit feedback
model = LightFM(loss='logistic')

# Train the model on the interactions matrix with item features and the individual ratings as weights
model.fit(interactions, item_features=item_features, sample_weight=weights, epochs=30, num_threads=2)

KeyboardInterrupt: 

In [7]:
all_cleaned.user_id.head()

0     16634.0
1     87141.0
2    169736.0
3    208406.0
4    230496.0
Name: user_id, dtype: object

In [8]:
# Define the recommend_books function
def recommend_books(model, interactions, user_id, user_mapping, item_mapping, isbn_info, num_recommendations=5):
    # Ensure the user_id is a string
    user_id = str(user_id)

    # Check if the user_id exists in the user mapping
    if user_id not in user_mapping:
        raise ValueError(f"User ID {user_id} is not found in the dataset.")

    # Get the internal index for the user_id
    user_idx = user_mapping[user_id]

    # Predict scores for all items for the given user
    scores = model.predict(user_idx, np.arange(interactions.shape[1]))

    # Get the indices of the top scores
    top_items = np.argsort(-scores)[:num_recommendations]

    # Map the indices back to ISBNs and fetch their title and author
    recommended_books = []
    for item in top_items:
        isbn = list(item_mapping.keys())[list(item_mapping.values()).index(item)]
        book_info = isbn_info.get(isbn, {'mod_book_title': 'Unknown Title', 'mod_book_author': 'Unknown Author'})
        recommended_books.append({'isbn': isbn, 'title': book_info['mod_book_title'], 'author': book_info['mod_book_author']})

    return recommended_books

In [9]:
# Example usage
user_id_to_recommend = '16634.0'  # Replace with an actual user_id from your dataset

try:
    # Generate recommendations
    recommended_books = recommend_books(model, interactions, user_id_to_recommend, user_mapping, item_mapping, isbn_info)
    
    # Display the recommended books with titles and authors
    for book in recommended_books:
        print(f"ISBN: {book['isbn']}, Title: {book['title']}, Author: {book['author']}")
except ValueError as e:
    print(e)

ISBN: 1883938813, Title: the ghost dance insurrection a jazzman novel, Author: jack random
ISBN: 1878093401, Title: marvin composes a tea and other humorous stories, Author: judith hunt
ISBN: 1878093398, Title: the ghostly bell ringer and other mysteries, Author: highlights for children
ISBN: 679721886, Title: the woman warrior memoirs of a girlhood among ghosts, Author: maxine hong kingston
ISBN: 1863733671, Title: on writing books for children, Author: jenny wagner
